# Modelo longitudinal basado en controlador difuso

Este modelo se generará a partir de los datos extraídos de los conductores. Para ello, se creará un controlador difuso ajustando su salida a través de un descenso del gradiente.

Primero se generará un controlador difuso a partir de los datos de conducción de **TODOS** los conductores. Luego se generará un controlador difuso para cada uno de ellos.

In [5]:
import os

import numpy as np
import pandas as pd
import tensorflow as tf

In [6]:
DATASETS_PATH = '/media/blazaid/Saca/Phd/data/datasets'
MOMENTS = 't-t5-t10-t20'
LEARNING_RATE = 0.01
TRAIN_STEPS = 1000
LOGS_STEPS = 100

In [15]:
training_df = pd.read_csv(os.path.join(DATASETS_PATH, 'cf-all-training-t-t5-t10-t20.csv'), index_col=False).astype(np.float32)

In [14]:
training_df.describe()

,Acceleration,Leader distance,Next TLS distance,Next TLS status,Relative speed,Speed to leader
count,1050.000000,1050.000000,1050.000000,1050.000000,1050.000000,1050.000000
mean,-0.001872,0.231711,0.356342,0.427143,0.417331,-0.090197
std,0.007998,0.115841,0.270940,0.470965,0.323679,0.377192
min,-0.023706,0.083861,0.005622,0.000000,0.000000,-10.073389
25%,-0.007871,0.143033,0.155141,0.000000,0.138784,-0.136922
50%,-0.001019,0.194978,0.282911,0.000000,0.344509,-0.024089
75%,0.002963,0.313926,0.469362,1.000000,0.725127,0.054234
max,0.025465,0.679816,1.000000,1.000000,1.072215,0.261483


In [ ]:
import collections

import numpy as np
import tensorflow as tf


IVar = collections.namedtuple('IVar', ('name', 'fuzzy_sets','domain'))
OVar = collections.namedtuple('OVar', ('name', 'values'))


def slope_asc(x, a, db):
    """ Tensor with the operation of an ascendent line.

    :param x: A tensor with the values to apply the function.
    :param a: The position where the lines stops being 0.
    :param db: The distance from the previous point (a) to the position where
        starts being 1.
    :return: A tensor of the same shape with the values of applying this
        function to the x tensor.
    """
    return tf.minimum(tf.maximum((x - a) / db, 0), 1)


def slope_desc(x, a, db):
    """ Tensor with the operation of an descendent line.

    :param x: A tensor with the values to apply the function.
    :param a: The position where the lines stops being 0.
    :param db: The distance from the previous point(a) to the position where
        starts being 1.
    :return: A tensor of the same shape with the values of applying this
        function to the x tensor.
    """
    return tf.minimum(tf.maximum((a - x) / db + 1, 0), 1)

def trapezoid(x, a, db, dc, dd):
    """ Returns a tensor with the operation of a trapezoidal mf.
    
    This operation will be a composition of two slopes like the ones defined
    in function `line`. This means that it is neccesary to initialize it with
    proper slopes values (i.e. first one with a positive value and the second
    one with a negative value).
    
    :param x: A tensor with the values to apply the function.
    :param a: The first (leftmost) point of the trapezoid.
    :param db: Distance from previous point (a) to the point where the
        trapezoid starts being 1 (point b). The function transforms it to a
        positive number so it is always greater or equal to 0.
    :param dc: Distance from previous point (b) to the point where the
        trapezoid stops being 1 (point c). The function transforms it to a
        positive number so it is always greater or equal to 0.
    :param dd: Distance from previous point (c) to the point where the
        trapezoid starts being 0 (point d). The function transforms it to a
        positive number so it is always greater or equal to 0.
    :return: A tensor of the same shape with the values of applying this
        function to the x tensor.
    """
    line_asc = (x - a) / db
    line_des = (a + db + dc - x) / dd + 1
    union = tf.minimum(line_asc, line_des)
    return tf.minimum(tf.maximum(union, 0), 1)


def log_asc(x, a, b, c):
    """ TBD """
    return 1 / (1 + tf.abs(a) * tf.exp(-b * (x - c)))


def log_desc(x, a, b, c):
    """ TBD """
    return 1 - log_asc(x, a, b, c)


def async_bell(x, a1, b1, c1, a2, b2, c2):
    """ TBD """
    return log_asc(x, a1, b1, c1) * log_desc(x, a2, b2, c2)


def fuzzification_graph(x, var_desc):
    """ TBD
    
    :param x: A tensor of shape (m, 1) wher m is each of the example values.
    :param var_desc: The description of this variable as a `IVar` tuple.
    :return: A tensor of shape (m, n) where m is the number of examples and n
        the number of fuzzy sets in this partition. The values will be the
        result of the fuzzification process where each column corresponds to
        this fuzzy set's membership function to the value.
    """
    with tf.variable_scope(var_desc.name):
        # The variables of this graph will be the shifts between points of the
        # membership functions. This shifts will be initialized to unfold all
        # the points equidistantly
        num_points = (var_desc.fuzzy_sets - 1) * 2
        lo, hi = min(var_desc.domain), max(var_desc.domain)
        shift_size = (hi - lo) / (num_points + 1)
        shifts = [
            tf.Variable(shift_size, name='s{}'.format(i))
            for i in range(num_points)
        ]
        
        # We define where the domain starts to define the rest of elements
        # via shifts
        base = tf.constant(name='b', value=lo, dtype=tf.float32)
        
        # Now, we create all the sets
        next_fs_starting_point = base + shifts[0]
        fuzzy_sets = []
        for i in range(var_desc.fuzzy_sets):
            # Depending on the index, create either desc, asc or trap fs.
            if i == 0:
                # First fuzzy set should be a descendent line
                fs = slope_desc(x, next_fs_starting_point, shifts[1])
            elif i == var_desc.fuzzy_sets - 1:
                # Last fuzzy set should be an ascendent line
                fs = slope_asc(x, next_fs_starting_point, shifts[-1])
            else:
                # Inner fuzzy sets should be a trapezoids
                shifts_to_apply = shifts[i * 2 - 1:i * 2 + 2]
                fs = trapezoid(x, next_fs_starting_point, *shifts_to_apply)
                next_fs_starting_point += shifts[(i-1)*2+1] + shifts[(i-1)*2+2]
            
            # Add this fs to the list of fuzzy_sets
            fuzzy_sets.append(fs)

        # Now concat all the fuzzy sets
        all_fuzzy_sets = tf.concat(fuzzy_sets, 0)

        # Return the created variable and the tensor with the fuzzifications
        # of the inputs
        return tf.transpose(tf.reshape(all_fuzzy_sets, (var_desc.fuzzy_sets, -1)))

def inference_graph(fuzzy_inputs, num_fuzzy_inputs, num_fuzzy_outputs):
    """ Creates the subgraph related to the fuzzy rules.
    
    :param f_inputs:
    :param num_f_outputs: The value of each singleton for each fuzzy output.
    """
    # First we create the cartesian product between all the fuzzy values of
    # each variable and then reduce them with the t-normof all the elements and make the
    # t-norm along the resulting elements (we call'em inferences).
    #
    # In the end inference will be the minimum of each combination between
    # fuzzy inputs, where the rows are the examples and the columns each
    # combination of fuzzy inputs.
    m = tf.shape(fuzzy_inputs[0])[0]  # The number of examples
    for fuzzy_input in fuzzy_inputs[1:]:
        inference = tf.minimum(fuzzy_inputs[0][:,None], fuzzy_input[:,:,None])
        inference = tf.reshape(inference, (m, -1))

    # Then, we create a set of weights of the size of the inference times the
    # number of fuzzy outputs. This implies that each of the inference will
    # have a weight over the final result. It can be change to binary values
    # to denote "this inference has/hasn't to do with this fuzzy output.
    num_combinations = np.prod(num_fuzzy_inputs)
    fuzzy_output_weights = tf.get_variable(
        'fuzzy_output_weights',
        shape=[num_fuzzy_outputs, 1, num_combinations],
        initializer=tf.contrib.layers.xavier_initializer(),
    )
    inference = tf.multiply(inference, tf.sigmoid(fuzzy_output_weights))

    # Now we reduce to the max the values of each of the outputs
    return tf.transpose(tf.reduce_max(inference, axis=2))


def defuzzification_graph(fuzzy_outputs, output_values):
    """ Media ponderada"""
    output_values = tf.constant(output_values, shape=[len(output_values)], dtype=tf.float32)
    num = tf.reduce_sum(tf.multiply(fuzzy_outputs, output_values), axis=1)

    return num[:,None]


def fuzzy_controller(i_vars, o_var):
    # Create the input placeholder for the controller and splitted to pass
    # each column to its fuzzification_graph
    ph_input = tf.placeholder(tf.float32, name='input', shape=[None, len(i_vars)])
    xs = tf.split(ph_input, num_or_size_splits=len(i_vars), axis=1)
    
    return xs

    # Generate each input variable fuzzification graph
    inputs = [fuzzification_graph(x, i_var) for x, i_var in zip(xs, i_vars)]

    # Generate the inference graph
    fuzzy_outputs = inference_graph(
        fuzzy_inputs=inputs,
        num_fuzzy_inputs=[i_var.fuzzy_sets for i_var in i_vars],
        num_fuzzy_outputs=len(o_var.values)
    )
    
    # Defuzzification graph
    defuzzification = defuzzification_graph(fuzzy_outputs, o_var.values)

    # Now we return the inputs as a placeholder with as much columns as
    # variables as a 1-column tensor.
    return ph_input, defuzzification

In [ ]:
tf.reset_default_graph()

# Input variables
leader_distance = IVar(name='LeaderDist', fuzzy_sets=3, domain=(0., 1.))
next_tls_distance = IVar(name='NextTlsDist', fuzzy_sets=3, domain=(0., 1.))
next_tls_status = IVar(name='NextTlsStatus', fuzzy_sets=3, domain=(0., 1.))
relative_speed = IVar(name='RelativeSpeed', fuzzy_sets=3, domain=(0., 2.))
speed_to_leader = IVar(name='SpeedToLeader', fuzzy_sets=3, domain=(-50., 50.))

# Output variable
acceleration = OVar(name='Acceleration', values=(-1, 1))

# Controller
x, y_hat = fuzzy_controller(
    i_vars=[leader_distance, next_tls_distance, next_tls_status, relative_speed, speed_to_leader],
    o_var=acceleration
)

# Training process
y = tf.placeholder(tf.float32)
cost = tf.reduce_mean(tf.squared_difference(y, y_hat))
train = tf.train.AdamOptimizer(LEARNING_RATE).minimize(cost)

inputs = training_df[['Leader distance', 'Next TLS distance', 'Next TLS status', 'Relative speed', 'Speed to leader']].values
output = training_df[['Acceleration']].values

init = tf.global_variables_initializer()
with tf.Session() as session:
    session.run(init)

    feed_dict = {x: inputs, y: output}
    for step in range(TRAIN_STEPS):
        session.run(train, feed_dict=feed_dict)
        if TRAIN_STEPS % LOGS_STEPS == 0:
            pass

In [ ]:
test_df = pd.read_csv(os.path.join(DATASETS_PATH, 'cf-miguel-validation-t-t5-t10-t20.csv'), index_col=False).astype(np.float32)